In [1]:
from pyomo.environ import ConcreteModel, Constraint, Objective, SolverFactory, TransformationFactory, Constraint, Var
from pyomo.network import Arc

In [2]:
from idaes.core import FlowsheetBlock
from idaes.unit_models import Mixer, HeatExchanger, Separator, GibbsReactor
# Methane combustion ideal package got CH4, H2O, CO, CO2, N2, NH3, O2
import idaes.property_models.activity_coeff_models.methane_combustion_ideal as thermo_props
#import idaes.property_models.activity_coeff_models.methane_combustion_ideal as reaction_props

In [3]:
import matplotlib.pyplot as plt

In [4]:
from idaes.unit_models.separator import SplittingType

### Building Base Flowsheet.

In [5]:
m = ConcreteModel()
m.fs = FlowsheetBlock(default={"dynamic": False})
m.fs.thermo_params = thermo_props.MethaneParameterBlock()

In [6]:
# Fuel ultilization (Uf): mole reductant consumed in FC per mole of reductant total
Uf = 0.8
# Air ultilization (Ua): mole of air consumed in FC per mole of air feed
Ua = 0.2
# Methane to steam ratio (MS): mole methane per mole water
MS = 2
# Feed:
# Reaction: 
# Reforming: CH4 + H2O -> CO + 3H2
# Water gas shift: CO + H2O -> CO2 + H2
# Methane combustion: CH4 + 2O2 -> CO2 + 2H2O
# Hydrogen combustion: H2 + 1/2O2 -> H2O
# Carbon monoxide combustion: CO + 1/2O2 -> CO2

n_CH4f = 10
print("mole of methane feed: "+str(n_CH4f)+" mole/s")
n_H2Of = n_CH4f*MS
print("mole of steam feed: "+str(n_H2Of)+" mole/s")
n_O2f = n_CH4f*Uf*2/Ua
n_N2f = n_O2f*0.79/0.21
print("mole of air feed: "+str(n_N2f+n_O2f)+" mole/s")

n_H2ex = 2
n_COex = n_CH4f*(1-Uf)*4-n_H2ex
n_CO2ex = n_CH4f-n_COex
n_H2Oex = n_H2Of+2*n_CH4f-n_H2ex
y_H2ex = n_H2ex/(n_H2ex + n_COex + n_CO2ex + n_H2Oex)
y_COex = n_COex/(n_H2ex + n_COex + n_CO2ex + n_H2Oex)
y_CO2ex = n_CO2ex/(n_H2ex + n_COex + n_CO2ex + n_H2Oex)
y_H2Oex = n_H2Oex/(n_H2ex + n_COex + n_CO2ex + n_H2Oex)

print("Anode exhaust: ")
print("y_H2ex: "+str(y_H2ex))
print("y_COex: "+str(y_COex))
print("y_CO2ex: "+str(y_CO2ex))
print("y_H2Oex: "+str(y_H2Oex))
print("Total mole/s: "+str(n_H2ex + n_COex + n_CO2ex + n_H2Oex))

n_N2ex = n_N2f
n_O2ex = n_O2f - n_CH4f*Uf*2
y_O2ex = n_O2ex/(n_O2ex+n_N2ex)
y_N2ex = n_N2ex/(n_O2ex+n_N2ex)
print("Cathode exhaust: ")
print("y_O2ex: "+str(y_O2ex))
print("y_N2ex: "+str(y_N2ex))
print("Total mole/s: "+str(n_O2ex+n_N2ex))

mole of methane feed: 10 mole/s
mole of steam feed: 20 mole/s
mole of air feed: 380.95238095238096 mole/s
Anode exhaust: 
y_H2ex: 0.04
y_COex: 0.11999999999999997
y_CO2ex: 0.08000000000000003
y_H2Oex: 0.76
Total mole/s: 50.0
Cathode exhaust: 
y_O2ex: 0.1753653444676409
y_N2ex: 0.824634655532359
Total mole/s: 364.95238095238096


### Cathode: A separator that splits exact the right amount of O2 out from the air stream

In [7]:
m.fs.sep1 = Separator(default={"dynamic": False,
                               "split_basis": SplittingType.componentFlow,
                               "property_package": m.fs.thermo_params})

In [8]:
m.fs.sep1.outlet_2_state

In [9]:
from idaes.core.control_volume0d import ControlVolume0DBlock

In [ ]:
m.fs.sep1.inlet.fix()

In [ ]:
m.fs.sep1.inlet.display()

In [ ]:
m.fs.sep1.outlet_1.mole_frac_comp[0.0,"CH4"].fix(0.0)
m.fs.sep1.outlet_1.mole_frac_comp[0.0,"CO"].fix(0.0)
m.fs.sep1.outlet_1.mole_frac_comp[0.0,"CO2"].fix(0.0)
m.fs.sep1.outlet_1.mole_frac_comp[0.0,"H2"].fix(0.0)
m.fs.sep1.outlet_1.mole_frac_comp[0.0,"H2O"].fix(0.0)
m.fs.sep1.outlet_1.mole_frac_comp[0.0,"N2"].fix(0.0)
m.fs.sep1.outlet_1.mole_frac_comp[0.0,"O2"].fix(1.0)
m.fs.sep1.outlet_1.flow_mol.fix(0.125)

In [ ]:
m.fs.sep1.inlet.display()

In [ ]:
from idaes.core.util.model_statistics import degrees_of_freedom as dof
dof(m)

In [ ]:
m.fs.sep1.initialize()

In [ ]:
solver = SolverFactory('ipopt')

In [ ]:
results = solver.solve(m, tee=True)

In [ ]:
m.fs.sep1.outlet_1.display()

In [ ]:
m.fs.sep1.outlet_2.display()

In [ ]:
m.fs.HX2 = HeatExchanger(default={"dynamic": False,
                                  "shell":{"property_package": m.fs.thermo_params},
                                  "tube":{"property_package": m.fs.thermo_params}})

In [ ]:
# COLD SIDE = TUBE : Air @ 25 C come in
m.fs.HX2.tube_inlet.flow_mol.fix(n_N2f+n_O2f)
m.fs.HX2.tube_inlet.mole_frac_comp[0.0,:].fix(0.0)
m.fs.HX2.tube_inlet.mole_frac_comp[0.0,"O2"].fix(0.21)
m.fs.HX2.tube_inlet.mole_frac_comp[0.0,"N2"].fix(0.79)
m.fs.HX2.tube_inlet.temperature.fix(25+273.15)
m.fs.HX2.tube_inlet.pressure.fix(101325)

In [ ]:
m.fs.HX2.tube_outlet.temperature.fix(700+273.15)

In [ ]:
m.fs.HX2.tube_inlet.display()

In [ ]:
m.fs.HX2.tube_outlet.display()

In [ ]:
# HOT SIDE = SHELL : Exhaust @ whatever temperature come in


In [ ]:
from idaes.core.util.model_statistics import degrees_of_freedom as dof
dof(m.fs.HX2)